In [72]:
import numpy as np
import pandas as pd
import matplotlib
import sqlite3
import sqlite3 as sql
import re

In [167]:
facr_df = pd.read_excel('C:\\Users\\micha\\Projects\\uhli\\data_z_klubu\\seznam-clenu.xls',index_col=None, na_values=['NA'])
conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
df_clen = pd.read_sql_query(" select * from moviebook_clen", conn)
df_platby = pd.read_sql_query(" select * from moviebook_prichozi_platby", conn)
conn.close()

In [168]:
df_platby.dtypes

id                       int64
datum                   object
objem                  float64
protiucet               object
kod_banky               object
zprava_pro_prijemce     object
poznamka                object
nazev_protiuctu         object
typ_platby              object
cislo_uctu_prichozi     object
jmeno                   object
prijmeni                object
facr_id                  int64
dtype: object

In [174]:
df_platby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2135 non-null   int64  
 1   datum                2135 non-null   object 
 2   objem                2135 non-null   float64
 3   protiucet            2135 non-null   object 
 4   kod_banky            2135 non-null   object 
 5   zprava_pro_prijemce  2135 non-null   object 
 6   poznamka             0 non-null      object 
 7   nazev_protiuctu      2135 non-null   object 
 8   typ_platby           2135 non-null   object 
 9   cislo_uctu_prichozi  0 non-null      object 
 10  jmeno                1418 non-null   object 
 11  prijmeni             1418 non-null   object 
 12  facr_id              2135 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 217.0+ KB


In [171]:
len(df_platby[df_platby.facr_id == 1])

717

In [173]:
df_platby[df_platby.facr_id==1]

,id,datum,objem,protiucet,kod_banky,zprava_pro_prijemce,poznamka,nazev_protiuctu,typ_platby,cislo_uctu_prichozi,jmeno,prijmeni,facr_id
0,1,04.01.2016,400.0,670100-2201092094,6210,"MATOUS PISL, POPLATEK ZA FOTBAL 1/2016",None,PETR PISL,Bezhotovostní příjem,None,None,None,1
1,2,04.01.2016,400.0,670100-2201092094,6210,"SIMON PISL, POPLATEK ZA FOTBAL 1/2016",None,PETR PISL,Bezhotovostní příjem,None,None,None,1
2,3,05.01.2016,400.0,27-9638310287,0100,JAN PLAINER FOTBAL LEDEN 2016 JAN PLAINER FOTB...,None,PLAINEROVÁ BLANKA,Bezhotovostní příjem,None,None,None,1
3,4,05.01.2016,200.0,27-9638310287,0100,ONDRA PLAINER FOTBAL LEDEN 201ONDRA PLAINER FO...,None,PLAINEROVÁ BLANKA,Bezhotovostní příjem,None,None,None,1
8,9,13.01.2016,400.0,533309,5500,Sebastian Hub 01/2016,None,Martin Hub,Bezhotovostní příjem,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2126,2127,09.11.2020,550.0,1174011193,0800,Theodor Zálešák 2005,None,Zálešáková Daniela,Bezhotovostní příjem,None,None,None,1
2130,2131,12.11.2020,550.0,614857113,0800,Sebastian_Ohem_2010,None,"Ohem Luděk,Ing.",Bezhotovostní příjem,None,None,None,1
2131,2132,12.11.2020,550.0,2265626163,0800,Jan Čenkovic 2014,None,Čenkovic Stanislav,Bezhotovostní příjem,None,None,None,1
2132,2133,13.11.2020,21600.0,59942,0800,,None,Up Česká republika s,Bezhotovostní příjem,None,None,None,1


In [45]:
df_clen.dtypes

id                           int64
email                       object
rc                          object
clenem_od                   object
active                       int64
jmeno                       object
prijmeni                    object
facr_id                      int64
var_symbol                   int64
klub_id                      int64
cislo_uctu                  object
telefonni_cislo             object
prijmeni_rodic              object
typ_clena                   object
ucet_kod_banky              object
ucet_nazev_protiuctu        object
ucet_poznamka               object
ucet_protiucet              object
ucet_zprava_pro_prijemce    object
rok_narozeni                object
ucet_protiucet2             object
narozen                     object
dtype: object

In [10]:
facr_df.dtypes

Členské ID          float64
Příjmení             object
Jméno                object
Rok narození          int64
Číslo klubu           int64
Název klubu          object
Členem od            object
První registrace     object
Aktivní člen         object
Stav emailu          object
Email                object
dtype: object

In [145]:
facr_df = (facr_df
    .rename({
        "Členské ID": "facr_id",
        "Příjmení":"prijmeni",
        "Jméno":"jmeno",
        "Členem od":"clenem_od",
        "Rok narození":"narozen_rok",   
        "Aktivní člen":"active",
        "Číslo klubu":"klub_id"
    }, axis="columns")
)
facr_df = (facr_df
    .assign(
        facr_id=facr_df['facr_id'].astype("string"),
        narozen_rok = facr_df['narozen_rok'].astype("string")
    )
)

In [127]:
# df_clen[(df_clen.narozen == '2010-01-01 00:00:00' ) &  (df_clen.prijmeni == 'Jantač')]
len(df_clen[(df_clen.jmeno == 'Alex') &  (df_clen.prijmeni == 'Kočandrle') & (pd.DatetimeIndex(df_clen.narozen).year == 2010)])


# is_there = df_clen['prijmeni'].str.lower().str.contains('koča',na=False)
# df_clen[ df_clen.prijmeni.str.lower().str.contains('koča',na=False) && df_clen.jmeno.str.lower().str.contains('alex',na=False)]
# is_koc = df_platby['zprava'].str.upper().str.contains('PLAIN',na=False)
# is_there.value_counts()


1

In [103]:
df_clen.dtypes

id                           int64
email                       object
rc                          object
clenem_od                   object
active                       int64
jmeno                       object
prijmeni                    object
facr_id                      int64
var_symbol                   int64
klub_id                      int64
cislo_uctu                  object
telefonni_cislo             object
prijmeni_rodic              object
typ_clena                   object
ucet_kod_banky              object
ucet_nazev_protiuctu        object
ucet_poznamka               object
ucet_protiucet              object
ucet_zprava_pro_prijemce    object
rok_narozeni                object
ucet_protiucet2             object
narozen                     object
dtype: object

In [108]:
facr_df.dtypes

facr_id             string
prijmeni            object
jmeno               object
narozen_rok         string
Číslo klubu          int64
Název klubu         object
clenem_od           object
První registrace    object
Aktivní člen        object
Stav emailu         object
Email               object
dtype: object

In [147]:
# columns = facr_df.columns.tolist()
# for index, row in facr_df.iterrows():
#     # Loop through columns
#     for col in columns:
#         cell = row[col]
#         # If we find it, print it out
#         print(f'cell {cell}')
#         if re.match("*kočan*", cell):
#             print(f'Found at Index: {index} Column: {col}')
conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
c = conn.cursor()

for index, row in facr_df.iterrows():
#     print(row['facr_id'], row['prijmeni'], row['jmeno'],  row['Email'], row['clenem_od'] )
    jmeno = row['jmeno']
    prijmeni = row['prijmeni']
    Email = row['Email']
    if pd.isnull(row['facr_id']):
        facr_id = 1
    else:
        facr_id = row['facr_id']

    if pd.isnull(row['clenem_od']):
        clenem_od = '01.01.1967'
    else:
        clenem_od = row['clenem_od']
        
    narozen_rok = row['narozen_rok']
    if row['active']=='Ano':
        active=1
    else:
        active=0
    klub_id = row['klub_id']
        
        
#     if len(df_clen[(df_clen.jmeno == row['jmeno']) & (df_clen.prijmeni == row['prijmeni'])  ]) == 1:        
    if len(df_clen[(df_clen.jmeno == row['jmeno']) & (df_clen.prijmeni == row['prijmeni']) & (pd.DatetimeIndex(df_clen.narozen).year == row['narozen_rok'])]) == 1:
        update_sql=f"UPDATE moviebook_clen set email='{Email}', facr_id={facr_id}, clenem_od='{clenem_od}', rok_narozeni='{narozen_rok}', active={active} where jmeno = '{jmeno}' and prijmeni = '{prijmeni}'"
    else:
        update_sql=f"insert into moviebook_clen (facr_id, email,  clenem_od, rok_narozeni, jmeno, prijmeni, active, var_symbol, klub_id) values({facr_id}, '{Email}','{clenem_od}','{narozen_rok}','{jmeno}','{prijmeni}', {active}, {facr_id}, {klub_id})"
#     c.execute(update_sql)
#     conn.commit()    
    print(update_sql)

    
    
    
conn.close()
    

UPDATE moviebook_clen set email='P.ADAMECKO@SEZNAM.CZ', facr_id=9070131.0, clenem_od='28.11.2014', rok_narozeni='2009', active=0 where jmeno = 'Jan' and prijmeni = 'Adamečko'
UPDATE moviebook_clen set email='sadameckova@seznam.cz', facr_id=11030619.0, clenem_od='21.02.2018', rok_narozeni='2011', active=0 where jmeno = 'Martin' and prijmeni = 'Adamečko'
UPDATE moviebook_clen set email='davidbares16@gmail.com', facr_id=82111297.0, clenem_od='10.10.2019', rok_narozeni='1982', active=0 where jmeno = 'David' and prijmeni = 'Bareš'
UPDATE moviebook_clen set email='lnimsova@gmail.com', facr_id=10121126.0, clenem_od='12.06.2019', rok_narozeni='2010', active=0 where jmeno = 'Karolína' and prijmeni = 'Barešová'
insert into moviebook_clen (facr_id, email,  clenem_od, rok_narozeni, jmeno, prijmeni, active, var_symbol, klub_id) values(84052349.0, 'lnimsova@gmail.com','09.10.2020','1984','Lucie','Barešová', 0, 84052349.0, 1050181)
UPDATE moviebook_clen set email='pravda.barton@gmail.com', facr_id=60

In [81]:
df_clen

,id,email,rc,clenem_od,active,jmeno,prijmeni,facr_id,var_symbol,klub_id,...,prijmeni_rodic,typ_clena,ucet_kod_banky,ucet_nazev_protiuctu,ucet_poznamka,ucet_protiucet,ucet_zprava_pro_prijemce,rok_narozeni,ucet_protiucet2,narozen
0,1,miko73@seznam.cz,12343331,None,1,Michal,Kocandrle,4,12343331,1050181,...,Jaromír Kočandrle,None,None,None,None,None,None,1111,None,1973-01-01 00:00:00
1,5,miko731@seznam.cz,12121212,None,1,Michal,Váňa,1,12121212,1050181,...,,,,,,,,1111,,1973-01-01 00:00:00
2,6,tobijas.kocandrle@gmail.com,123567,None,1,Tobiáš,Kočandrle,10,123567,1050181,...,,,,,,,,1111,,1973-01-01 00:00:00
3,8,None,1310161534,07.02.2020,1,Lukáš,Baumgardner,1,1310161534,1050181,...,None,None,None,None,None,None,None,None,,2013-01-01 00:00:00
4,9,None,1310268253,19.02.2019,1,Adam,Debnár,13100166,13100166,1050181,...,None,None,None,None,None,None,None,None,,2013-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,198,e.heinzova@volny.cz,None,None,1,Petr,Heinz,1,198,1050181,...,HEINZOVA EDITA,None,0300,None,None,125965877,MP_Matyas_Heinz,None,,1973-01-01 00:00:00
188,199,e.heinzova@volny.cz,None,None,1,Edita,Heinzova,1,199,1050181,...,HEINZOVA EDITA,None,0300,None,None,125965877,vstup do SK US - E. Heinzova,None,,1973-01-01 00:00:00
189,200,jan.vondracek@bidfood.cz,None,None,1,JAN,VONDRACEK,1,200,1050181,...,VONDRÁČKOVÁ EVA,None,0100,None,None,35-5471610297,VONDRACEK,None,,1973-01-01 00:00:00
190,201,alena.meliskova@sond-or.cz,1208220277,None,1,Matej,Komárek,0,1208220277,1050181,...,Meliskova Alena,None,0300,None,None,272972102,MP_Matej_Komarek,None,None,1973-01-01 00:00:00


In [ ]:
conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
c_clen = conn.cursor()
c_platby = conn.cursor()
# parameter1 = "Urxova 458/8, Praha, 18600"    
c_platby.execute("select protiucet, kod_banky, zprava_pro_prijemce from moviebook_prichozi_platby;")
# clens.execute("")
platby_arr = c_platby.fetchall()
# print(rows)
new_rc = 0
for p_ar in platby_arr:
    # print (p_ar)
    q_clen=f"select var_symbol, jmeno, prijmeni from moviebook_clen WHERE ucet_protiucet = '{p_ar[0]}' and ucet_kod_banky = '{p_ar[1]}'" 
    print(q_clen)
    c_clen.execute(q_clen)
    clen_arr = c_clen.fetchall()
    if len (clen_arr) > 0:
        q_update = f"update moviebook_prichozi_platby set facr_id='{clen_arr[0][0]}',jmeno='{clen_arr[0][1]}',prijmeni='{clen_arr[0][2]}' where protiucet = '{p_ar[0]}' and kod_banky='{p_ar[1]}' and (facr_id is null or facr_id = 1)"
        print ("q_update - ", q_update)
#         c_platby.execute(q_update)
#         conn.commit()    
        
conn.close()

        

In [161]:
#hleda cislo uctu a kod banky v doslych platbach a updatuje celny cisly uctu

conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
c_clen = conn.cursor()
c_platby = conn.cursor()
# parameter1 = "Urxova 458/8, Praha, 18600"    
c_clen.execute("select jmeno, prijmeni, var_symbol from moviebook_clen where ucet_kod_banky IS NULL or ucet_kod_banky = '' or cislo_uctu IS NULL or cislo_uctu=''")
# clens.execute("")
clen_arr = c_clen.fetchall()
# print(rows)
new_rc = 0
for c_ar in clen_arr:
    # print (p_ar)
    q_platba=f"select protiucet, kod_banky from moviebook_prichozi_platby where zprava_pro_prijemce like '%{c_ar[0]}%' and zprava_pro_prijemce like '%{c_ar[1]}%'" 
#     print(q_clen)
    c_platby.execute(q_platba)
    platby_arr = c_platby.fetchall()
    if len (platby_arr) > 0 and :
        q_update = f"update moviebook_clen set ucet_kod_banky='{platby_arr[0][1]}',cislo_uctu='{platby_arr[0][0]}' where jmeno = '{c_ar[0]}' and prijmeni='{c_ar[1]}' and var_symbol='{c_ar[2]}'"
        print ("q_update - ", q_update)
        c_platby.execute(q_update)
        conn.commit()         
conn.close()


q_update -  update moviebook_clen set ucet_kod_banky='0800',cislo_uctu='2259804103' where jmeno = 'Tomáš' and prijmeni='Vacík' and var_symbol='13020634'
q_update -  update moviebook_clen set ucet_kod_banky='6210',cislo_uctu='670100-2200315938' where jmeno = 'Jim' and prijmeni='Bráza' and var_symbol='8090455'
q_update -  update moviebook_clen set ucet_kod_banky='0100',cislo_uctu='19-8471050297' where jmeno = 'Šimon' and prijmeni='Přáda' and var_symbol='8020974'
q_update -  update moviebook_clen set ucet_kod_banky='2010',cislo_uctu='2600138184' where jmeno = 'Jan' and prijmeni='Sedelmayer' and var_symbol='8031205'
q_update -  update moviebook_clen set ucet_kod_banky='5500',cislo_uctu='518575028' where jmeno = 'Samuel' and prijmeni='Šmerda' and var_symbol='1408150326'
q_update -  update moviebook_clen set ucet_kod_banky='2010',cislo_uctu='2900118951' where jmeno = 'Teodor' and prijmeni='Cír' and var_symbol='134'
q_update -  update moviebook_clen set ucet_kod_banky='0800',cislo_uctu='13065

In [165]:
conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
c_clen = conn.cursor()
c_platby = conn.cursor()
# parameter1 = "Urxova 458/8, Praha, 18600"    
c_platby.execute("select protiucet, kod_banky, zprava_pro_prijemce from moviebook_prichozi_platby;")
# clens.execute("")
platby_arr = c_platby.fetchall()
# print(rows)
new_rc = 0
for p_ar in platby_arr:
    # print (p_ar)
    q_clen=f"select var_symbol, jmeno, prijmeni from moviebook_clen WHERE ucet_protiucet = '{p_ar[0]}' and ucet_kod_banky = '{p_ar[1]}'" 
#     print(q_clen)
    c_clen.execute(q_clen)
    clen_arr = c_clen.fetchall()
    if len (clen_arr) > 0:
        q_update = f"update moviebook_prichozi_platby set facr_id='{clen_arr[0][0]}',jmeno='{clen_arr[0][1]}',prijmeni='{clen_arr[0][2]}' where protiucet = '{p_ar[0]}' and kod_banky='{p_ar[1]}' and (facr_id is null or facr_id = 1)"
        print ("q_update - ", q_update)
        c_platby.execute(q_update)
        conn.commit()
conn.close()

q_update -  update moviebook_prichozi_platby set facr_id='9040274',jmeno='Šimon',prijmeni='Douša' where protiucet = '51-1683520287' and kod_banky='0100' and (facr_id is null or facr_id = 1)
q_update -  update moviebook_prichozi_platby set facr_id='149',jmeno='Filip',prijmeni='Trněný' where protiucet = '2102186087' and kod_banky='2700' and (facr_id is null or facr_id = 1)
q_update -  update moviebook_prichozi_platby set facr_id='9090086',jmeno='Marek',prijmeni='Cibulka' where protiucet = '1551743163' and kod_banky='0800' and (facr_id is null or facr_id = 1)
q_update -  update moviebook_prichozi_platby set facr_id='9040274',jmeno='Šimon',prijmeni='Douša' where protiucet = '51-1683520287' and kod_banky='0100' and (facr_id is null or facr_id = 1)
q_update -  update moviebook_prichozi_platby set facr_id='143',jmeno='David',prijmeni='Senolt' where protiucet = '230982820' and kod_banky='0300' and (facr_id is null or facr_id = 1)
q_update -  update moviebook_prichozi_platby set facr_id='147',j

In [182]:
conn = sql.connect('C:\\Users\\micha\\Projects\\uhli\\db.sqlite3')
c_clen = conn.cursor()
c_platby = conn.cursor()
# parameter1 = "Urxova 458/8, Praha, 18600"    
c_clen.execute("select jmeno, prijmeni, var_symbol, cislo_uctu, ucet_kod_banky from moviebook_clen where var_symbol IS not NULL and  not var_symbol = '1' ")
# clens.execute("")
clen_arr = c_clen.fetchall()
# print(rows)
new_rc = 0
for c_ar in clen_arr:
    # print (p_ar)
    q_platba=f"select id, protiucet, kod_banky, facr_id from moviebook_prichozi_platby where zprava_pro_prijemce like '%{c_ar[0]}%' and zprava_pro_prijemce like '%{c_ar[1]}%'" 
#     print(q_clen)
    c_platby.execute(q_platba)
    platby_arr = c_platby.fetchall()
    for pl_ar in platby_arr:
        if pl_ar[3] != c_ar[2]:
            q_update = f"update moviebook_prichozi_platby set jmeno = '{c_ar[0]}', prijmeni = '{c_ar[1]}', facr_id= {c_ar[2]} where id = '{pl_ar[0]}'"
            print ("q_update - ", q_update)
#             c_platby.execute(q_update)
#             conn.commit()         
conn.close()


q_update -  update moviebook_prichozi_platby set jmeno = 'Lukáš', prijmeni = 'Novotný', facr_id= 13080308 where id = '2073'
q_update -  update moviebook_prichozi_platby set jmeno = 'Lukáš', prijmeni = 'Novotný', facr_id= 13080308 where id = '2075'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1608'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1670'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1718'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1798'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1841'
q_update -  update moviebook_prichozi_platby set jmeno = 'Jakub', prijmeni = 'Tajtl', facr_id= 13050342 where id = '1884'
q_update -  update m